<a href="https://colab.research.google.com/github/zeze79/kakao-clone/blob/master/explore_11/E11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import glob
import os

import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 1. 데이터 읽어오기

In [10]:

txt_file_path = '/content/drive/MyDrive/Colab Notebooks/explore/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['They say get ready for the revolution', "I think it's time we find some sorta solution", "Somebody's caught up in the endless pollution"]


## Step 2. 데이터 정제

In [11]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.


<start> this is sample sentence . <end>


In [12]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]


['<start> they say get ready for the revolution <end>',
 '<start> i think it s time we find some sorta solution <end>',
 '<start> somebody s caught up in the endless pollution <end>',
 '<start> they need to wake up , stop living illusions i know you need to hear this <end>',
 '<start> why won t somebody feel this <end>',
 '<start> this is my wish that we all feel connected <end>',
 '<start> this is my wish that nobodies neglected be like a rocket baby <end>',
 '<start> be like a rocket take off <end>',
 '<start> just fly , away ay , ay <end>',
 '<start> to find your space take off <end>']

In [13]:
#tf keras txt tokenizer
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15) 

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  45  68 ...   0   0   0]
 [  2   5 126 ...   0   0   0]
 [  2 265  16 ...   0   0   0]
 ...
 [  2  31  21 ...   1   3   0]
 [  2 338  21 ...   3   0   0]
 [  2  42 130 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fc18cecf748>


In [14]:
print(tensor[:3, :10])

[[   2   45   68   43  297   28    6 2109    3    0]
 [   2    5  126   11   16   73   23  204   99 3845]
 [   2  265   16  635   29   14    6 2963    1    3]]


In [15]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [16]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])


[   2   45   68   43  297   28    6 2109    3    0    0    0    0    0]
[  45   68   43  297   28    6 2109    3    0    0    0    0    0    0]


In [17]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [18]:
#tensors = np.array(tensors)

NameError: ignored

## Step 3. 평가데이터셋 분리

In [19]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

In [20]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


## Step 4. 인공지능 만들기

In [21]:
tokenizer.num_words

12000

### model

In [22]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [23]:
for src_sample, tgt_sample in dataset.take(1):
    break
model(src_sample)


<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 4.60201773e-05, -9.66208681e-05, -1.09552733e-04, ...,
          1.61692049e-04, -4.44294710e-05,  2.16669534e-04],
        [ 2.38142980e-04,  1.05256513e-04, -9.17766156e-05, ...,
          6.80893718e-04, -2.10979066e-04,  2.94500613e-04],
        [ 3.47258610e-04,  2.27052325e-04,  3.51493873e-05, ...,
          1.03231263e-03, -2.80695589e-04,  2.55632855e-04],
        ...,
        [-1.30599481e-03, -8.11637379e-04, -7.23538033e-05, ...,
          1.94792898e-04,  9.02261294e-04, -3.77925142e-04],
        [-1.51122292e-03, -8.62084678e-04, -3.57041630e-04, ...,
         -3.32167721e-04,  7.34721543e-04, -2.11167484e-04],
        [-1.70101190e-03, -8.77295097e-04, -6.17565995e-04, ...,
         -8.17824563e-04,  5.03106334e-04, -5.11878352e-05]],

       [[ 4.60201773e-05, -9.66208681e-05, -1.09552733e-04, ...,
          1.61692049e-04, -4.44294710e-05,  2.16669534e-04],
        [-1.67492362e-05, -9.46159562e-05, -1

In [24]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


### loss & optimizer

In [25]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

optimizer = tf.keras.optimizers.Adam()


### trainning

In [26]:
model.compile(loss=loss, optimizer=optimizer)
model.fit(x=enc_train, y=dec_train, validation_data=(enc_val, dec_val), batch_size=BATCH_SIZE, epochs=10)


Epoch 1/10
550/550 [==============================] - 107s 195ms/step - loss: 3.7023 - val_loss: 3.3355
Epoch 2/10
550/550 [==============================] - 113s 206ms/step - loss: 3.2046 - val_loss: 3.1336
Epoch 3/10
550/550 [==============================] - 117s 212ms/step - loss: 3.0155 - val_loss: 3.0028
Epoch 4/10
550/550 [==============================] - 118s 214ms/step - loss: 2.8707 - val_loss: 2.9114
Epoch 5/10
550/550 [==============================] - 117s 212ms/step - loss: 2.7485 - val_loss: 2.8415
Epoch 6/10
550/550 [==============================] - 117s 213ms/step - loss: 2.6380 - val_loss: 2.7875
Epoch 7/10
550/550 [==============================] - 117s 213ms/step - loss: 2.5345 - val_loss: 2.7353
Epoch 8/10
550/550 [==============================] - 118s 214ms/step - loss: 2.4368 - val_loss: 2.6962
Epoch 9/10
550/550 [==============================] - 117s 213ms/step - loss: 2.3447 - val_loss: 2.6613
Epoch 10/10
550/550 [==============================] - 117s 213m

## Step5. 평가

In [27]:
def generate_text(model, tokenizer, init_sentence='<start> i love', max_len=20):
    
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index['<end>']
    
    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:,-1]
        
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        if predict_word.numpy()[0] == end_token:break
        if test_tensor.shape[1] >= max_len:break
    
    generated = ""
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    
    return generated

generate_text(model, tokenizer, init_sentence='<start> i love', max_len=20)


'<start> i love you , i m gonna be <end> '

In [29]:
generate_text(model, tokenizer, init_sentence='<start> i hate', max_len=20)

'<start> i hate the way you lie <end> '

In [30]:
generate_text(model, tokenizer, init_sentence='<start> if you', max_len=20)

'<start> if you want me , i m not gonna crack my mind <end> '

In [31]:
generate_text(model, tokenizer, init_sentence="<start> show me the ", max_len=20)


'<start> show me the place <end> '

In [32]:
generate_text(model, tokenizer, init_sentence="<start> i am", max_len=20)

'<start> i am not throwing away my shot <end> '